# Andvaranaut tutorial

## Forward Module (Input distributions specified)

### Latin Hypercube Sampling
Import items form forward module as well as utils module

In [ ]:
from andvaranaut.forward import *
from andvaranaut.utils import *

In [ ]:
# Magic features for development purposes
%load_ext autoreload
%autoreload 2

User supplies target function, which takes a 1D numpy array of nx inputs and returns a 1D numpy array of ny outputs. They must also supply a list of univariate distributions from scipy stats for each of the nx inputs.

In [ ]:
# Example target function (2 inputs, 2 outputs)
# A more complex target function will produce an input file, 
# execute external code, and perform post-processing on output
def test_fun(x):
  x1,x2 = x
  return np.array([x1+x2,10000*x1**2*x2**2])

# Input variable probability distributions
import scipy.stats as st
sample_space = [st.uniform(loc=0,scale=2),\
                st.norm(loc=1,scale=0.5)]

In [ ]:
# Latin hypercube class instance, with correct arguments
l = lhc(nx=2,ny=2,dists=sample_space,target=test_fun)

Sampling makes use of the latin_random function from py-design

In [ ]:
# Sample input distributions by LHC and evaluate target function
l.sample(nsamps=4)
print(l.x)
print(l.y)

Parallel execution makes use of the ray package. This also works with SLURM submission if a SLURM script calls a python script with these commands in. (Tutorial will be added at a later date)

In [ ]:
# Can also execute target function evaluation in parallel
l.sample(nsamps=4,parallel=True,nproc=4)
print(l.x)
print(l.y)

Plotting output distributions makes use of the kdeplot function from seaborn

In [ ]:
# Plot output distributions based on kernel density estimation
l.y_dist()

In [ ]:
# Optionally delete n samples
# Default is deletion by closest sample to a coarse LHC of number of samples for deletion
l.del_samples(ndels=2,method='coarse_lhc')
print(l.x)
print(l.y,'\n')
# Can also delete by random indexing
l.del_samples(ndels=2,method='random')
print(l.x)
print(l.y,'\n')
# or by specific data indexes
l.del_samples(method='specific',idx=[0,1])
print(l.x)
print(l.y,'\n')

### Gaussian process surrogate

In addition to the arguments provided to the lhc class, there are additional arguments for a list of classes which handle conversion and reversion of the x and y datasets, respectively. These are necessary for optimising surrogate performance, and usually consist of transforming bounded ranges on inputs and outputs to unbounded. Normalisations to get numbers O(1) are also useful and can be implemented either here or within the target function.

These conversion/reversion arguments are optional, and can be left blank if desired. Standard methods are provided in andvaranaut.utils, with the logarithm and uniform classes shown below for clarity on the format. A user can define their own class in this format, as long as any additional arguments like the distribution function in uniform are packaged into partial functions within the class.

In [ ]:
# Convert positive values to unbounded with logarithm
def log_con(y):
  return np.log10(y)
# Revert logarithm with power
def log_rev(y):
  return np.power(10,y)
class logarithm:
  def __init__(self):
    self.con = log_con # Conversion function
    self.rev = log_rev # Reversion function

from functools import partial
# Convert uniform dist samples into standard uniform 
def std_uniform(x,dist):
  intv = dist.interval(1.0)
  x = (x-intv[0])/(intv[1]-intv[0])
  return x
# Revert to original uniform distributions
def uniform_rev(x,dist):
  intv = dist.interval(1.0)
  x = x*(intv[1]-intv[0])+intv[0]
  return x
class uniform:
  def __init__(self,dist):
    self.con = partial(std_uniform,dist=dist)
    self.rev = partial(uniform_rev,dist=dist)

In [ ]:
# Define lists of conversion/reversion classes for each x and y variable
xconrevs = [logit_logistic(sample_space[0]),normal(sample_space[1])]
yconrevs = [None,nonneg()]
# Instance of gp
g = gp(nx=2,ny=2,dists=sample_space,target=test_fun,xconrevs=xconrevs,yconrevs=yconrevs)

In [ ]:
g.sample(5)
print(g.x)
print(g.y)
print(g.xc)
print(g.yc)

## Utils module

### Save and load objects

In [ ]:
# Save lhc class including datasets
save_object(obj=l,fname='lhc_tut.pickle')

In [ ]:
# Load lhc class
l = load_object(fname='lhc_tut.pickle')
print(l.x)
print(l.y)